In [ ]:
#pip install -q langchain
#pip install -q pymupdf
#pip install -q huggingface-hub
#pip install -q faiss-cpu
#pip install -q sentence-transformers
#pip install -q openai

In [27]:
import os 
os.environ["TOKENIZERS_PARALLELISM"] = "true"
import langchain
import textwrap
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.llms import OpenAI
import warnings
warnings.filterwarnings('ignore')

In [28]:
# Funcao para carregar o pdf
def carrega_pdf(file_path):
    #Cria uma instancia da classe PyMuPDFLoader passando o caminho
    loader = PyMuPDFLoader(file_path=file_path)
    
    #Usa o metodo 'load' do objeto 'loader' para carregar o conteúdo do PDF
    #Isso retorna um objeto ou uma estrutura de dados contendo as paginas do PDF com o seu conteúdo.
    docs = loader.load()
    
    #Retorna o conteúd carregado do PDF
    return docs

In [29]:
# Função para dividir os documentos em vários pedações (chuncks)
def split_docs(documents, chunck_size = 1000, chunk_overlap = 20):
    #Cria uma instância da classe RecursiveCharacterTextSplitter
    #Esta classa divide textos longo em pedações menores (chunks)
    #'chunk_size' define o tamanho de cada pedaço e 'chunck_overlap' define a sobreposição entre pedações consecutivos
    text_splitter = RecursiveCharacterTextSplitter(chunck_size, chunk_overlap = chunk_overlap, length_function=len, is_separator_regex=False)
    
    #Utiliza o método 'split_documents' do objeto 'text_spliter' para dividir o documento fornecido.
    #'documents' é uma variável que contém o texto ou conjunto de textos a serem divididos
    chunks = text_splitter.split_documents(documents = documents)
    
    #Retorna os pedaçõs de texto resultantes da divisão
    return chunks

In [30]:
#Carrega o modelo de embeddings
def carrega_embedding_model(model_path, normalize_embedding=True):
    #Retorna uma instância da classe HuggingFaceEmbeddings
    # 'model_name' é o identificador do modelo de embeddings a ser carregado
    # 'model_kwargs' é um dicionário de argumentos adicionais para a configuração do modelo, neste caso, definindo o dispositivo para 'cpu'
    # 'encode_kwargs' é um dicionário de argumentos para o método de codificação, aqui especificando se os embeddings devem ser normalizados.
    
    return HuggingFaceEmbeddings(model_name=model_path, model_kwargs= {'device': 'cpu'}, encode_kwargs={'normalize_embedding': normalize_embedding})

In [31]:
#Função para criar embeddings
def cria_embeddings(chunks, embedding_model, storing_path = "modelo/vectorstore"):
    #Cria um 'vectorstore' (um índice FAISS) a partir dos documentos fornecidos
    #'chunks' é a lista de segmentos de texto e 'embedding_model' é o modelo de embedding utilizado para converter texto em embeddings.
    vectorstore = FAISS.from_documents(chunks, embedding_model)
    
    #Salva o 'vectorstore' criado em um caminho local especificado por 'storing_path'
    #Isso permite a persistência do índice FAISS para uso futuro.
    vectorstore.save_local(storing_path)
    
    #Retorna o 'vectorstore' criado, que contém os embeddings e pode ser usado para operações de busca e comparação de similaridade
    return vectorstore

In [32]:
#Criando a chain
def load_qa_chain(retriever, llm, prompt):
    #Retorna uma instancia da classe RetrievalQA
    #Esta função lida com a cadeia de processo envolvidos em um sistema de Question Answering(QA)
    #'llm' refere-se ao modelo de linguagem de grande escala (como um modelo GPT ou BERT)
    #'retriever' é um componente usado para recuperar informações relevantes (como um mecanismo de busca ou um retriever de documentos)
    #'chain_type' define o tipo de cadeia ou estratégia usada no processo de QA. Aqui, está definido como "stuff", um placeholder para um tipo real.
    #'return_source_documents': um booleano que, quando True, indica que os documentos fonte (ou seja, os documentos de onde as respostas são extraídas) devem ser retornados juntamente com as respostas
    # 'chain_type_kwargs: é um dicionário de argumento adicionais específicos para o tipo de cadeia escolhido. Aqui está passando 'prompt' como argumento
    return RetrievalQA.from_chain_type(llm= llm,
                                       retriever=retriever,
                                       chain_type="stuff",
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt})

In [33]:
#Função para obter as respostas do LLM (Large Language Model)
def get_response(query, chain):
    #Invoca a 'chain' (cadeia de processamento, um pipeline de Question Aswering) com 'query' fornecida
    #'chain' é uma função que recebe uma consulta e retorna uma resposta, utilizando o LLM
    response = chain({'query': query})
    
    #Utiliza a biblioteca textwrap para formatar a resposta. 'textwrap.fill' quebra o texto da resposta em linhas de largura especificada (100 caracteres neste caso),
    #tornando mais fácil a leitura em ambientes como o Jupyter Notebook
    wrapped_text = textwrap.fill(response['result'], width=100)
    
    #Imprime o texto formatado
    print(wrapped_text)

In [34]:
# Definindo a API da Open AI
llm_api = OpenAI(openai_api_key="")

In [35]:
# Carrega o Modelo Embedding
embed = carrega_embedding_model(model_path = "all-MiniLM-L6-v2")

In [36]:
# Carrega o arquivo PDF
#docs = carrega_pdf(file_path = "../projeto_especial_ia_sql/processos_ds.pdf")
#docs = carrega_pdf(file_path = "../projeto_especial_ia_sql/processos_metodos_ds.pdf")
docs = carrega_pdf(file_path = "../projeto_especial_ia_sql/metodos_ds.pdf")

In [37]:
# Divide o arquivo em Chunks
documents = split_docs(documents = docs)

TypeError: 'int' object is not subscriptable

In [ ]:
# Cria vectorstore
vectorstore = cria_embeddings(documents, embed)

In [ ]:
# Converte o vectorstore em um retriever
retriever = vectorstore.as_retriever()

In [ ]:
# Cria template
template = """
### System:
Você é um assistente pessoal. Você tem que responder as perguntas do usuário \
usando apenas o contexto fornecido a você. Se você não sabe a resposta, \
apenas diga que você não sabe. Não tente inventar uma resposta.

### Context:
{context}

### User:
{question}

### Response: 
"""

In [ ]:
# Criando prompt a partir do template
prompt = PromptTemplate.from_template(template)

In [ ]:
# Criando a chain
dsa_chain = load_qa_chain(retriever, llm_api, prompt)

In [ ]:
# Interagindo com o assistente
get_response("Quais são os processos de trabalho de um cientista de dados executa?", dsa_chain)

get_response("Quais métodos um cientista de dados utiliza?", dsa_chain)

# Fim